# HW 9: 自駕車安全 (資訊安全)

2020.11.27

## Tutorial

- [TA placehold]()
- [Cybersecurity: Crash Course Computer Science #31 (12:29)](https://www.youtube.com/watch?v=bPVaOlJ6ln0)
- [The Internet: Encryption & Public Keys (06:39)](https://www.youtube.com/watch?v=ZghMPWGXexs)


## Objective

- Learn how to sniffer self network packet
- Understand what is cybersecurity
- Understand why need to computer security


## Problem

在前面 `Lab7 自駕車間的溝通 (網路工程)`，我們可以透過網路與小鴨車交換資料  
但透過 `TCP` 與 `UDP` 在網路傳輸資料是安全的嗎？


## Requirement

學習使用開源的 網路封包分析工具 `Wireshark`


## Notice

- deadline: 2020.12.04 before class